In [133]:
import pandas as pd
import zipfile
import time


# Defined Functions

In [204]:
# function to get the stats for the half - returns stats and game winner

def half_stats(gameNum,gameWinner,gameYear,play_df):
    half_csv=play_df[play_df["Game Code"]==gameNum].reset_index()
    team1,team2 = half_csv["Offense Team Code"].unique()
    tc1,tc2=team1,team2

    #create dictionary for stats
    stats={
        team1:{'TEAM CODE':tc1,
               'GAME CODE':gameNum,
               'YEAR':gameYear,
               'RUSH':0,
               'OFFENSE RUSH YARDS':0,
               'PASS':0,
               'INC PASS':0,
               'OFFENSE PASS YARDS':0,
               'OFFENSE POINTS':0,
               'PENALTY':0,
               'PENALTY YARDS':0,
               'DEFENSE RUSH YARDS':0,
               'DEFENSE PASS YARDS':0,
               'DEFENSE POINTS':0,
               'RESULT':0
        },
        team2:{'TEAM CODE':tc2,
               'GAME CODE':gameNum,
               'YEAR':gameYear,
               'RUSH':0,
               'OFFENSE RUSH YARDS':0,
               'PASS':0,
               'INC PASS':0,
               'OFFENSE PASS YARDS':0,
               'OFFENSE POINTS':0,
               'PENALTY':0,
               'PENALTY YARDS':0,
               'DEFENSE RUSH YARDS':0,
               'DEFENSE PASS YARDS':0,
               'DEFENSE POINTS':0,
               'RESULT':0
        }
    }
    
    # Put in game winner in dictionary
    if(gameWinner != 'None'):
        stats[gameWinner]['RESULT']=1

    numPlays=len(half_csv)
    for index, row in half_csv.iterrows():
        if(index+1<numPlays):
            # Get the current team on offense
            team=half_csv.iloc[index]['Offense Team Code']
            
            # Get the play type
            playType=half_csv.iloc[index]['Play Type']

            # Beginning and ending yardage
            spot1=half_csv.iloc[index]['Spot']
            spot2=half_csv.iloc[index+1]['Spot']
            numYards=spot1-spot2

            # Update Dictionary depending on play type
            if(playType=='RUSH'):
                stats[team][playType]=stats[team][playType]+1
                stats[team]['OFFENSE RUSH YARDS']=stats[team]['OFFENSE RUSH YARDS']+numYards
            if(playType=='PASS'):
                stats[team][playType]=stats[team][playType]+1
                stats[team]['OFFENSE PASS YARDS']=stats[team]['OFFENSE PASS YARDS']+numYards
                if(numYards==0):
                    stats[team]['INC PASS']=stats[team]['INC PASS']+1
            if(playType=='PENALTY'):
                # If the penalty has positve numYards, it was against offense
                if(numYards>0):
                    stats[team][playType]=stats[team][playType]+1
                    stats[team]['PENALTY YARDS']=stats[team]['PENALTY YARDS']+numYards
                # If the penalty has negative numYards, it was against offense
                else:
                    team=half_csv.iloc[index]['Defense Team Code']
                    stats[team][playType]=stats[team][playType]+1
                    stats[team]['PENALTY YARDS']=stats[team]['PENALTY YARDS']-numYards

    # Defensive rushing and passing yards = Offensinve rishing and passing yards of opponent
    stats[team1]['DEFENSE RUSH YARDS']=stats[team2]['OFFENSE RUSH YARDS']
    stats[team2]['DEFENSE RUSH YARDS']=stats[team1]['OFFENSE RUSH YARDS']
    stats[team1]['DEFENSE PASS YARDS']=stats[team2]['OFFENSE PASS YARDS']
    stats[team2]['DEFENSE PASS YARDS']=stats[team1]['OFFENSE PASS YARDS']

    # Get the halftime scores for each team, save to dictionary
    team=half_csv.tail(1)['Offense Team Code'].values[0]
    teamPoints=half_csv.tail(1)['Offense Points'].values[0]
    stats[team]['OFFENSE POINTS']=half_csv.tail(1)['Offense Points'].values[0]
    team=half_csv.tail(1)['Defense Team Code'].values[0]
    teamPoints=half_csv.tail(1)['Defense Points'].values[0]
    stats[team]['OFFENSE POINTS']=half_csv.tail(1)['Defense Points'].values[0]

    stats[team1]['DEFENSE POINTS']=stats[team2]['OFFENSE POINTS']
    stats[team2]['DEFENSE POINTS']=stats[team1]['OFFENSE POINTS']

    # Make new dictionary keys so that you don't need team code to access data
    stats[1]=stats[team1]
    stats[2]=stats[team2]

    return (stats)

In [205]:
# function to find the game winner - return team code

def getWinner(gameNum,play4th):
    lastLine=play4th[play4th["Game Code"]==gameNum]
    # Apparently there was at least one game without a 4th quarter?!?
    # Let's use a conditional...
    if(len(lastLine)>0):
        lastLine=play4th[play4th["Game Code"]==gameNum].tail(1)
        score1=lastLine['Offense Points'].values[0]
        score2=lastLine['Defense Points'].values[0]
        if (score1 > score2):
            winningTeam=lastLine['Offense Team Code'].values[0]
        elif (score1 < score2):
            winningTeam=lastLine['Defense Team Code'].values[0]
        else:
            # We got ourselves a tie!!!
            winningTeam='NONE'
    else:
        winningTeam='NONE'

    return (winningTeam)


#   Read in play files and get stats for each half

In [206]:
years=('2008','2009','2010','2011','2012','2013')
path=".//kaggleGameData//play"

# Create DataFrame to put the stats in
first_half_stats_df=pd.DataFrame()

for year in years:
    # start monitor timer
    t0=time.time()

    # Read in the play file
    fileName=".//kaggleData//play"+year+".csv.zip"
    playAll=pd.read_csv(fileName)

    # Throw out all plays that occur in second half to use with stats function
    play1stHalf=playAll[playAll["Period Number"]<3]
    # Use the 4th quarter (and beyond) for game results
    play4th=playAll[playAll["Period Number"]>3]
    
    # Get the game codes
    gameCodes=play1stHalf['Game Code'].unique()

    # Get first-half stats for each game
    for gameCode in gameCodes:
        # find the game winner
        gameWinner=getWinner(gameCode,play4th)
        if(gameWinner!='NONE'):
            # get the 1st half stats
            stats=half_stats(gameCode,gameWinner,year,play1stHalf)
            # Append results to the dataframe
            first_half_stats_df=first_half_stats_df.append(stats[1],ignore_index='True')
            first_half_stats_df=first_half_stats_df.append(stats[2],ignore_index='True')

    print(year+' finished in ',(time.time()-t0),' seconds')


2008 finished in  61.80194163322449  seconds
2009 finished in  61.469823360443115  seconds
2010 finished in  60.17508840560913  seconds
2011 finished in  60.31176972389221  seconds
2012 finished in  62.999391078948975  seconds
2013 finished in  67.1000964641571  seconds


In [208]:
first_half_stats_df.head()

,DEFENSE PASS YARDS,DEFENSE POINTS,DEFENSE RUSH YARDS,GAME CODE,INC PASS,OFFENSE PASS YARDS,OFFENSE POINTS,OFFENSE RUSH YARDS,PASS,PENALTY,PENALTY YARDS,RESULT,RUSH,TEAM CODE,YEAR
0,108.0,7.0,72.0,5.000047e+14,2.0,180.0,28.0,105.0,13.0,4.0,28.0,1.0,11.0,47.0,2008
1,180.0,28.0,105.0,5.000047e+14,12.0,108.0,7.0,72.0,23.0,2.0,20.0,0.0,21.0,500.0,2008
2,35.0,6.0,75.0,7.490051e+14,6.0,138.0,20.0,63.0,24.0,1.0,5.0,1.0,22.0,749.0,2008
3,138.0,20.0,63.0,7.490051e+14,9.0,35.0,6.0,75.0,16.0,0.0,0.0,0.0,18.0,51.0,2008
4,113.0,14.0,88.0,7.040086e+14,5.0,142.0,21.0,102.0,12.0,0.0,0.0,1.0,18.0,86.0,2008


# Clean up the first half stats Dataframe and print to csv

In [209]:
half_stats_df=first_half_stats_df.copy()
# Change all the stats to integers since the opriginal file didn't have decimals
# and all codes must be integers anyway
half_stats_df=first_half_stats_df.copy()
half_stats_df = half_stats_df.astype('int64')

# Let's re-order those columns
half_stats_df=half_stats_df[['YEAR','GAME CODE','TEAM CODE','RESULT','OFFENSE POINTS','DEFENSE POINTS','RUSH','OFFENSE RUSH YARDS','PASS',"INC PASS","OFFENSE PASS YARDS",'DEFENSE RUSH YARDS','DEFENSE PASS YARDS','PENALTY','PENALTY YARDS']]

half_stats_df.head()


,YEAR,GAME CODE,TEAM CODE,RESULT,OFFENSE POINTS,DEFENSE POINTS,RUSH,OFFENSE RUSH YARDS,PASS,INC PASS,OFFENSE PASS YARDS,DEFENSE RUSH YARDS,DEFENSE PASS YARDS,PENALTY,PENALTY YARDS
0,2008,500004720080828,47,1,28,7,11,105,13,2,180,72,108,4,28
1,2008,500004720080828,500,0,7,28,21,72,23,12,108,105,180,2,20
2,2008,749005120080828,749,1,20,6,22,63,24,6,138,75,35,1,5
3,2008,749005120080828,51,0,6,20,18,75,16,9,35,63,138,0,0
4,2008,704008620080828,86,1,21,14,18,102,12,5,142,88,113,0,0


In [210]:
# Write to csv
fileName='.//halfResults//halfStats'
half_stats_df.to_csv(fileName+'.csv',index=False)
# Write to zipped csv
compression_opts = dict(method='zip',
                        archive_name=fileName+'.csv')  
half_stats_df.to_csv(fileName+'.zip', index=False, compression=compression_opts)